In [ ]:
from icecube import icetray, dataclasses, dataio, recclasses
import numpy as np
import glob
import json

files = glob.glob("./search_data/2*/Alerts*.i3")
files.sort()
print(files)
dir_path = '/home/jovyan/alert_cat_v2/realtime_hese/resources/data/'

from realtime_hese.python.HESE_alerts_v2 import *


In [ ]:
event_dict = {}
for file in files:
    ofile = dataio.I3File(file)
    while ofile.more():
        frame = ofile.pop_physics()
        fm_hese = frame['FilterMask']['HESEFilter_15']
        if fm_hese.condition_passed:
            cqtot = frame['HESE_CausalQTot'].value
            ldir = round(frame['OnlineL2_SplineMPE_DirectHitsB'].dir_track_length,5)
            zen = round(frame["OnlineL2_SplineMPE"].dir.zenith, 5)
            cos_zen = np.cos(zen)
            azi = round(frame["OnlineL2_SplineMPE"].dir.azimuth, 5)
            truncatedEnergyName = 'OnlineL2_SplineMPE_TruncatedEnergy_AllDOMS_Muon'
            truncatedEnergy = round(frame[truncatedEnergyName].energy,5) if frame.Has(truncatedEnergyName) else None
            llh_ratio =round(frame["HESE_llhratio"].value, 5)
            header = frame['I3EventHeader']
            mjd = header.start_time.mod_julian_day_double
            run_id = header.run_id
            event_id = header.event_id
            event_sub_id = header.sub_event_id
            ra, dec = astro.dir_to_equa(zen, azi, mjd)
            #print(header.start_time.mod_julian_day_double, header.run_id, header.event_id, ra, dec)
            #print('HESE cqtot:',cqtot, ldir, cos_zen, truncatedEnergy, llh_ratio)
            pass_precut = (cqtot > 6000.) and (ldir > 200.)
            pass_loose  = pass_precut and np.asscalar(HESE_track_selection_soft(cqtot, llh_ratio, cos_zen))
            pass_tight  = pass_precut and np.asscalar(HESE_track_selection_hard(cqtot, llh_ratio, cos_zen))
            #print ('Alert:', pass_loose, pass_tight)
            err_50, err_90 = locate_angular_uncertainty(ldir, pass_tight, pass_loose)
            sig_ness   = locate_signalness(cqtot, cos_zen, pass_tight, pass_loose)
            exp_rate = locate_bkg_rate(cqtot, pass_tight, pass_loose)
            nu_energy = locate_Enu(truncatedEnergy, pass_tight, pass_loose)
            select = 'None'
            if pass_tight:
                select = 'Gold'
            else:
                if pass_loose:
                    select = 'Bronze'
            overlaps = ''
            if frame.Has("Alert_JSON"):
                gfu_alert = json.loads(frame['Alert_JSON'].value)
                if gfu_alert['pass_tight']:
                    overlaps = '+GFU Gold'
                else:
                    if gfu_alert['pass_loose']:
                        overlaps= '+GFU Bronze'
            if pass_loose or pass_tight:
                #print ('Alert:', pass_loose, pass_tight)
                #print(header.start_time.mod_julian_day_double, header.run_id, header.event_id)
                #print('HESE cqtot:',cqtot, ldir, cos_zen, truncatedEnergy, llh_ratio)
                print('File:',file)
                print(f"Run: {run_id:6} Event: {event_id:8} Subevent:{event_sub_id:3} MJD: {mjd:0.2f} ",\
                      f"RA: {np.rad2deg(ra):0.2f} DEC: {np.rad2deg(dec):0.2f} ERR50: {np.rad2deg(err_50):0.2f} ERR90: {np.rad2deg(err_90):0.2f} ",\
                      f"E_muon:{(truncatedEnergy/1000.0):0.2f}  E_nu:{(nu_energy/1000.0):0.2f} Sgns: {sig_ness:0.2f}",\
                      f"CQTOT: {cqtot:7.2f} Selection: {select:8}  {overlaps:10}")
                key = str(run_id) + '_' + str(event_id)
                print(key)
                if run_id != 123770:
                    event_dict[key] = {
                        'MJD' : mjd,
                        'RA' : np.rad2deg(ra),
                        'DEC' : np.rad2deg(dec),
                        'ERR50' : np.rad2deg(err_50),
                        'ERR90' : np.rad2deg(err_90),
                        'E_NU' : nu_energy,
                        'SGNS' : sig_ness,
                        'FAR' : exp_rate,
                        'SELECTION' : select
                    }
#print(event_dict)
for key in event_dict.keys():
            print(key, event_dict[key])
with open('hese_events.txt', 'w') as outfile:
    json.dump(event_dict, outfile)
                
                #print(frame)
            #evaluate hese filter
            
#        if phys.Has("Alert_JSON"):
#            header = phys['I3EventHeader']
#            #print(header.start_time.mod_julian_day_double)
#            gfu_alert = json.loads(phys['Alert_JSON'].value)
#            select = 'None'
#            if gfu_alert['pass_tight']:
#                select = 'Gold'
#            else:
#                if gfu_alert['pass_loose']:
#                    select = 'Bronze'
#            print(f"Run: {gfu_alert['run_id']:6} Event: {gfu_alert['event_id']:8} Subevent:{header.sub_event_id:3} MJD: {header.start_time.mod_julian_day_double:0.2f} ",\
#                  f"RA: {np.rad2deg(gfu_alert['ra']):0.2f} DEC: {np.rad2deg(gfu_alert['dec']):0.2f} ERR50: {np.rad2deg(gfu_alert['err_50']):0.2f} ERR90: {np.rad2deg(gfu_alert['err_90']):0.2f} ",\
#                  f"E_muon:{(gfu_alert['E_muon']/1000.0):0.2f}  E_nu:{(gfu_alert['E_nu_peak']/1000.0):0.2f} Sgns: {gfu_alert['signalness']:0.2f}",\
#                  f"Selection: {select:10}")
#    #print(gfu_alert)
